## Analysis of Quast reports

Sergio Álvarez-Pérez, 2020

In [ ]:
from scipy import stats
from collections import Counter
from matplotlib.ticker import MaxNLocator
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
if not os.path.exists('/home/sergio/TFM1/reports/'): # include here your preferred path
    os.mkdir('/home/sergio/TFM1/reports/')
    
if not os.path.exists('/home/sergio/TFM1/reports/quast'):
    os.mkdir('/home/sergio/TFM1/reports/quast')

In [ ]:
# Quality checks: assemblies with N50 <25000 bp and/or >500 Ns per 100000 bases will be considered to be of poor quality

path = '/home/sergio/TFM1/quast_out/' # path with the outputs of Quast

qc_file = open('/home/sergio/TFM1/reports/quast/quast_quality_checks.txt', 'w') # quality checks report
qc_file.write("QUALITY CHECKS\n")
qc_file.write("-------------------------------------------------\n\n")


assembly_list = []
contigs_0_list = []
contigs_1000_list = []
contigs_5000_list = []
contigs_10000_list = []
contigs_25000_list = []
contigs_50000_list = []
genome_size_list = []
length_0_list = []
length_1000_list = []
length_5000_list = []
length_10000_list = []
length_25000_list = []
length_50000_list = []
contigs_list = []
largest_contig_list = []
total_length_list = []
GC_list = []
N50_list = []
N75_list = []
L50_list = []
L75_list = []
Ns_list = []
          

for foldername in os.listdir(path):
    for file in os.listdir(path + '/' + foldername):
        if file == "transposed_report.tsv":
            f = open(path + '/' + foldername + '/' + file, "r")
            lines = f.readlines()
            assembly, contigs_0, contigs_1000, contigs_5000, contigs_10000, contigs_25000, contigs_50000, length_0, length_1000, length_5000, length_10000, length_25000, length_50000, contigs, largest_contig, total_length, GC, N50, N75, L50, L75, Ns = lines[1].split("\t")
            if assembly not in ['Pasteurella_multocida_ASM75427v1','Photobacterium_phosphoreum_ASM302627v1','Vibrio_cholerae_ASM62164v1']: # to remove non-Enterobacterales (i.e. outgroup) species
                assembly_list.append(assembly)
                contigs_0_list.append(int(contigs_0))
                contigs_1000_list.append(int(contigs_1000))
                contigs_5000_list.append(int(contigs_5000))
                contigs_10000_list.append(int(contigs_10000))
                contigs_25000_list.append(int(contigs_25000))
                contigs_50000_list.append(int(contigs_50000))
                genome_size_list.append(int(length_0)/1e6)
                length_0_list.append(int(length_0))
                length_1000_list.append(int(length_1000))
                length_5000_list.append(int(length_5000))
                length_10000_list.append(int(length_10000))
                length_25000_list.append(int(length_25000))
                length_50000_list.append(int(length_50000))
                contigs_list.append(int(contigs))
                largest_contig_list.append(int(largest_contig))
                total_length_list.append(int(total_length))
                GC_list.append(float(GC))
                N50_list.append(int(N50))
                N75_list.append(int(N75))
                L50_list.append(int(L50))
                L75_list.append(int(L75))
                Ns_list.append(float(Ns))
            
            if int(N50) < 25000: # N50 <25000 bp
                qc_file.write(">>> Low N50 value for genome " + assembly + " --> N50 = " + str(N50) +  "\n")
            
            if float(Ns) > 500: # Ns per 100000 bases >500
                qc_file.write(">>> High number of Ns for genome " + assembly + " --> Ns per 100 kbp = " + str(Ns) +  "\n")
            
            if int(N50) >= 25000 and float(Ns) <= 500:
                qc_file.write(">>> QC check PASSED for genome " + assembly + " !!!\n")

            f.close()
            
qc_file.write("\n**************** END OF THE FILE ****************")
qc_file.close()


mylists = {'assembly': assembly_list, '# contigs (>= 0 bp)': contigs_0_list, '# contigs (>= 1000 bp)': contigs_1000_list,
           '# contigs (>= 5000 bp)': contigs_5000_list, '# contigs (>= 10000 bp)': contigs_10000_list, '# contigs (>= 25000 bp)': contigs_25000_list,
           '# contigs (>= 50000 bp)': contigs_50000_list, 'Genome size (Mb)': genome_size_list, 'Total length (>= 0 bp)': length_0_list, 'Total length (>= 1000 bp)': length_1000_list,
           'Total length (>= 5000 bp)': length_5000_list, 'Total length (>= 10000 bp)': length_10000_list, 'Total length (>= 25000 bp)': length_25000_list,
           'Total length (>= 50000 bp)': length_50000_list, '# contigs': contigs_list, 'Largest contig': largest_contig_list,
           'Total length': total_length_list, '%GC': GC_list, 'N50': N50_list, 'N75': N75_list, 'L50': L50_list,
           'L75': L75_list, '# Ns per 100 kbp': Ns_list}

In [ ]:
# To create a pandas dataframe from the dictionary obtained above and generate a .csv report

df = pd.DataFrame.from_dict(mylists)
pd.DataFrame.to_csv(df, path_or_buf = '/home/sergio/TFM1/reports/quast/quast_results_per_species.tsv', sep = "\t", index = False)
df

In [ ]:
# To generate a report with some basic statistics for each parameter and barplots in .pdf format

results_file = open('/home/sergio/TFM1/reports/quast/quast_summary_statistics.tsv', 'w')
results_file.write("Parameter\tn\tmean\ts.d.\tmedian\tmode\n")

parameters = ['# contigs', 'Genome size (Mb)', '%GC', 'N50', 'L50', '# Ns per 100 kbp']


for k, v in mylists.items():
    if k != 'assembly':
        results_file.write(k + "\t" + str(len(v)) + "\t" + str(np.round(np.average(v),2)) + "\t" + str(np.round(np.std(v),2)) + "\t" + str(np.round(np.median(v),2)) + "\t" + str(stats.mode(v)[0][0]) + "\n")
        
        if k in parameters:
            ax = df.hist(column=k, bins='auto', grid=False, xlabelsize=14, xrot=45, ylabelsize=14, align='mid', rwidth=0.5)
            ax = ax[0]
            for x in ax:
                x.set_title("")
                x.set_ylabel("No. genomes", labelpad=10, weight='bold', size=14)
                x.set_xlabel(k, labelpad=10, weight='bold', size=14)
                x.yaxis.set_major_locator(MaxNLocator(integer=True))
    
                plt.savefig("/home/sergio/TFM1/reports/quast/plot_QUAST_" + k + ".pdf", bbox_inches='tight')
        
results_file.close()

In [ ]:
# The following commands will generate box-and-whiskers plots for some key parameters (genome size and GC content)
# Then, a swarm plot will be place on top of each panel. The position of Rosenbergiella genomes will be indicated by red dots

df.insert(1, 'Genus', 'Other')
df

In [ ]:
df.loc[(df.assembly.str.startswith('Rosenbergiella'),'Genus')]='Rosenbergiella'
df

In [ ]:
mypalette = ['grey','red']
sns.set(font_scale=1.2, style='white') 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5))
g1 = sns.boxplot(x='Genome size (Mb)', data=df, color='lightsteelblue', fliersize=0, ax=ax1)
g1 = sns.swarmplot(x='Genome size (Mb)', y=[""]*len(df), hue='Genus', palette=mypalette, data=df, ax=ax1)
g1.legend_.remove()
g2 = sns.boxplot(x='%GC', data=df, color='lightgreen', fliersize=0, ax=ax2)
g2 = sns.swarmplot(x='%GC', y=[""]*len(df), hue='Genus', palette=mypalette, data=df, ax=ax2)
g2.legend_.remove()

fig.savefig('/home/sergio/TFM1/reports/quast/QUAST_boxplots.pdf', bbox_inches='tight')